In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

PATH = os.getcwd()
for _ in range(2):
    PATH = os.path.dirname(PATH)
if PATH not in sys.path:
    sys.path.append(PATH)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from FactorAnalysis.week1.helpers.data_processing import read_from_db, read_table_names
from FactorAnalysis.week2.helpers.information_analysis import (
    factor_analysis_all,
    read_in_weights,
    combine_factors
)

In [4]:
sns.set_theme()
warnings.filterwarnings("ignore")

## Load Factors and Prices

In [5]:
start_date = "20190101"
db_factors = "data/factors.db"

factors = read_from_db("factors_all_stocks", db_factors)

In [6]:
factors

total_mv        pe        pb  turnover_rate  \
trade_date ts_code                                                  
2019-01-02 000001.SZ  0.083321 -0.325763 -0.798117      -0.304775   
           000002.SZ  0.083321 -0.206204 -0.480028      -0.135223   
           000006.SZ  0.702998  0.950617  0.348110       0.968434   
           000008.SZ  0.581733  2.102146  1.447103       1.740295   
           000009.SZ -1.733993 -1.530981 -0.199528      -0.993448   
...                        ...       ...       ...            ...   
2023-12-29 688777.SH -0.222965 -0.200384 -0.401124      -0.377340   
           688778.SH -0.771222 -0.461310 -0.824449      -0.793190   
           688779.SH -2.473606 -1.008549 -0.815579      -2.495824   
           688819.SH -0.702957 -0.805802 -0.532022      -0.969003   
           688981.SH -0.288936 -0.496714 -0.855544      -0.488154   

                      reversal_rate  volatility       roe  netprofit_yoy  \
trade_date ts_code                                                         
2019-01-02 000001.SZ      -0.220369   -0.228010 -0.114297      -0.176164   
           000002.SZ      -0.212351   -0.267856  0.082697      -0.013247   
           000006.SZ       0.935827    0.895779  0.971955       0.924936   
           000008.SZ       1.717541    1.623287  1.257846       1.665371   
           000009.SZ      -1.231527   -1.224059 -1.038154      -1.369969   
...                             ...         ...       ...            ...   
2023-12-29 688777.SH      -0.401381   -0.601752 -0.462780      -0.513021   
           688778.SH      -1.053647   -0.790082 -0.731853      -0.252151   
           688779.SH      -2.495088   -2.391307 -1.870585      -1.014213   
           688819.SH      -0.932444   -0.917142 -1.146047      -1.004502   
           688981.SH      -0.389428   -0.677466 -0.352826      -0.234297   

                        or_yoy  assets_yoy  equity_yoy  gross_profit_margin  \
trade_date ts_code                                                            
2019-01-02 000001.SZ -0.229718   -0.195412   -0.480238            -0.452793   
           000002.SZ -0.157485    0.395986   -0.234699             0.000424   
           000006.SZ  1.020208    0.952265    0.835304             0.935789   
           000008.SZ  1.823129    1.882059    1.221329             0.970596   
           000009.SZ -1.496202   -1.467010   -0.804477            -1.445616   
...                        ...         ...         ...                  ...   
2023-12-29 688777.SH -0.516439   -0.453111   -0.517085            -0.501424   
           688778.SH -0.394839   -0.397358   -0.006730            -0.377000   
           688779.SH -1.034186   -1.091583   -1.868308            -1.058373   
           688819.SH -1.003178   -1.048111   -0.887566            -1.157365   
           688981.SH -0.401160   -0.479728   -0.506171            -0.393868   

                      operating_profit_margin  net_profit_margin  \
trade_date ts_code                                                 
2019-01-02 000001.SZ                -0.459658          -0.418976   
           000002.SZ                -0.013598          -0.093235   
           000006.SZ                 0.927483           0.917090   
           000008.SZ                 0.962309           0.960389   
           000009.SZ                -1.485266          -0.988157   
...                                       ...                ...   
2023-12-29 688777.SH                -0.498649          -0.493062   
           688778.SH                -0.384726          -0.243073   
           688779.SH                -1.072313          -1.065926   
           688819.SH                -1.153338          -1.143297   
           688981.SH                -0.391102          -0.382208   

                      operating_cash_flow_to_net_income  \
trade_date ts_code                                        
2019-01-02 000001.SZ                          -0.339621   
           000002.SZ                         

## Combine Factors

In [7]:
weights_df = read_in_weights()
weights_df

,total_mv,pe,pb,turnover_rate,reversal_rate,volatility,roe,netprofit_yoy,or_yoy,assets_yoy,equity_yoy,gross_profit_margin,operating_profit_margin,net_profit_margin,operating_cash_flow_to_net_income,operating_cash_flow_to_revenue,current_ratio,cash_current_liability_ratio,cash_liability_ratio,long_term_liability_operating_cash_flow_ratio
IC_10D,0.009776,-0.051812,-0.044281,-0.051301,-0.050766,-0.053280,-0.051306,-0.059359,-0.057962,-0.049469,-0.052405,-0.054424,-0.054296,-0.054258,-0.050817,-0.051871,-0.051079,-0.050721,-0.050630,-0.050187
IC_1D,-0.034544,0.015960,-0.027533,-0.001884,0.013930,0.031507,0.064962,0.042942,0.050698,0.063379,0.075235,0.072201,0.072734,0.072649,0.056136,0.057112,0.073284,0.057406,0.058035,0.057869
IC_5D,0.009992,-0.053345,-0.047652,-0.053597,-0.051911,-0.054109,-0.050562,-0.059921,-0.057970,-0.049046,-0.051495,-0.053387,-0.053221,-0.053230,-0.050254,-0.051198,-0.049645,-0.050000,-0.049914,-0.049551
IC_rank_10D,0.002840,-0.052204,-0.037167,-0.050214,-0.051809,-0.050973,-0.051992,-0.059046,-0.057271,-0.050459,-0.054318,-0.057049,-0.056888,-0.056735,-0.052073,-0.052747,-0.052716,-0.051121,-0.051106,-0.051274
IC_rank_1D,-0.054004,-0.015140,-0.037045,-0.037198,-0.048657,0.056118,0.058804,0.026329,0.039092,0.056153,0.073730,0.062461,0.062850,0.064868,0.045737,0.045888,0.077475,0.046603,0.047204,0.044642
IC_rank_5D,0.000176,-0.053822,-0.038029,-0.051686,-0.053875,-0.050050,-0.051600,-0.060027,-0.057734,-0.050333,-0.054324,-0.056734,-0.056615,-0.056413,-0.051807,-0.052404,-0.051754,-0.050641,-0.050703,-0.051272
IR_1D,-0.047885,0.017805,-0.035626,-0.001742,0.012235,0.031099,0.061497,0.043727,0.051684,0.061734,0.079614,0.071372,0.071881,0.072245,0.053052,0.054026,0.071479,0.054050,0.054710,0.052537
IR_5D,0.011885,-0.060314,-0.060179,-0.050225,-0.046205,-0.053442,-0.047926,-0.060683,-0.058947,-0.048613,-0.055147,-0.053312,-0.053180,-0.053489,-0.047802,-0.048779,-0.049213,-0.047765,-0.047717,-0.045177
IR_10D,0.009690,-0.059052,-0.055229,-0.048546,-0.045835,-0.053469,-0.047651,-0.060908,-0.058582,-0.048174,-0.056067,-0.054877,-0.054820,-0.054963,-0.048457,-0.049916,-0.050117,-0.048746,-0.048626,-0.046273
IR_rank_1D,-0.079035,-0.016747,-0.046068,-0.033551,-0.042234,0.053487,0.055259,0.025996,0.038317,0.053777,0.076778,0.062761,0.063126,0.065431,0.042381,0.042628,0.075782,0.042941,0.043555,0.040147


In [8]:
db_out_filename = "data/combined_factors.db"
combined_factors = combine_factors(factors, weights_df, db_out_filename)
combined_factors

IC_10D     IC_1D     IC_5D  IC_rank_10D  IC_rank_1D  \
trade_date ts_code                                                            
2019-01-02 000001.SZ  0.316205 -0.268799  0.317371     0.315399   -0.196979   
           000002.SZ  0.158128 -0.113168  0.159362     0.155529   -0.077551   
           000006.SZ -0.877584  0.820225 -0.875525    -0.892974    0.593089   
           000008.SZ -1.444015  1.212432 -1.448062    -1.453559    0.813689   
           000009.SZ  1.036315 -0.949642  1.033794     1.062858   -0.662007   
...                        ...       ...       ...          ...         ...   
2023-12-29 688777.SH  0.416214 -0.379571  0.416015     0.420395   -0.283712   
           688778.SH  0.529804 -0.396453  0.532321     0.534836   -0.224171   
           688779.SH  1.670957 -1.474135  1.668533     1.702817   -1.001081   
           688819.SH  0.938793 -0.882261  0.936565     0.954246   -0.641553   
           688981.SH  0.398772 -0.303984  0.401417     0.398776   -0.202384   

                      IC_rank_5D     IR_1D     IR_5D    IR_10D  IR_rank_1D  \
trade_date ts_code                                                           
2019-01-02 000001.SZ    0.316223 -0.258811  0.325115  0.323956   -0.188321   
           000002.SZ    0.155852 -0.106590  0.162463  0.161536   -0.072177   
           000006.SZ   -0.897035  0.789079 -0.865639 -0.871567    0.557461   
           000008.SZ   -1.462231  1.163067 -1.444954 -1.445167    0.760419   
           000009.SZ    1.071049 -0.906732  1.024029  1.033029   -0.603039   
...                          ...       ...       ...       ...         ...   
2023-12-29 688777.SH    0.421933 -0.365246  0.413921  0.415758   -0.267945   
           688778.SH    0.539466 -0.361628  0.523818  0.523908   -0.187682   
           688779.SH    1.712639 -1.386152  1.628355  1.641645   -0.900213   
           688819.SH    0.958226 -0.847408  0.926697  0.933033   -0.602803   
           688981.SH    0.401417 -0.286616  0.406980  0.405402   -0.181647   

                      IR_rank_5D  IR_rank_10D  
trade_date ts_code                             
2019-01-02 000001.SZ    0.323839     0.322782  
           000002.SZ    0.158095     0.157922  
           000006.SZ   -0.892046    -0.887210  
           000008.SZ   -1.460244    -1.451426  
           000009.SZ    1.071496     1.061855  
...                          ...          ...  
2023-12-29 688777.SH    0.420455     0.418707  
           688778.SH    0.532140     0.527770  
           688779.SH    1.680079     1.670020  
           688819.SH    0.952799     0.948213  
           688981.SH    0.406441     0.403887  

[1398271 rows x 12 columns]

In [9]:
combined_factors_reloaded = read_from_db("combined_factors", db_out_filename)
combined_factors_reloaded

IC_10D     IC_1D     IC_5D  IC_rank_10D  IC_rank_1D  \
trade_date ts_code                                                            
2019-01-02 000001.SZ  0.316205 -0.268799  0.317371     0.315399   -0.196979   
           000002.SZ  0.158128 -0.113168  0.159362     0.155529   -0.077551   
           000006.SZ -0.877584  0.820225 -0.875525    -0.892974    0.593089   
           000008.SZ -1.444015  1.212432 -1.448062    -1.453559    0.813689   
           000009.SZ  1.036315 -0.949642  1.033794     1.062858   -0.662007   
...                        ...       ...       ...          ...         ...   
2023-12-29 688777.SH  0.416214 -0.379571  0.416015     0.420395   -0.283712   
           688778.SH  0.529804 -0.396453  0.532321     0.534836   -0.224171   
           688779.SH  1.670957 -1.474135  1.668533     1.702817   -1.001081   
           688819.SH  0.938793 -0.882261  0.936565     0.954246   -0.641553   
           688981.SH  0.398772 -0.303984  0.401417     0.398776   -0.202384   

                      IC_rank_5D     IR_1D     IR_5D    IR_10D  IR_rank_1D  \
trade_date ts_code                                                           
2019-01-02 000001.SZ    0.316223 -0.258811  0.325115  0.323956   -0.188321   
           000002.SZ    0.155852 -0.106590  0.162463  0.161536   -0.072177   
           000006.SZ   -0.897035  0.789079 -0.865639 -0.871567    0.557461   
           000008.SZ   -1.462231  1.163067 -1.444954 -1.445167    0.760419   
           000009.SZ    1.071049 -0.906732  1.024029  1.033029   -0.603039   
...                          ...       ...       ...       ...         ...   
2023-12-29 688777.SH    0.421933 -0.365246  0.413921  0.415758   -0.267945   
           688778.SH    0.539466 -0.361628  0.523818  0.523908   -0.187682   
           688779.SH    1.712639 -1.386152  1.628355  1.641645   -0.900213   
           688819.SH    0.958226 -0.847408  0.926697  0.933033   -0.602803   
           688981.SH    0.401417 -0.286616  0.406980  0.405402   -0.181647   

                      IR_rank_5D  IR_rank_10D  
trade_date ts_code                             
2019-01-02 000001.SZ    0.323839     0.322782  
           000002.SZ    0.158095     0.157922  
           000006.SZ   -0.892046    -0.887210  
           000008.SZ   -1.460244    -1.451426  
           000009.SZ    1.071496     1.061855  
...                          ...          ...  
2023-12-29 688777.SH    0.420455     0.418707  
           688778.SH    0.532140     0.527770  
           688779.SH    1.680079     1.670020  
           688819.SH    0.952799     0.948213  
           688981.SH    0.406441     0.403887  

[1398271 rows x 12 columns]